In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd                # Data exploration, manipulation functions
import matplotlib.pyplot as plt
import seaborn as sns              # Plots
import re                          # Regular Expression

In [ ]:
forest_train = pd.read_csv("/kaggle/input/forest-cover-type-prediction/train.csv")  # Import the train data set

In [ ]:
forest_train.head()

In [ ]:
forest_train.info()        

In [ ]:
rtrain,coltrain  = forest_train.shape  #Dimensions of training data set
rtrain

In [ ]:
plt.figure(figsize=(20,10))     
sns.swarmplot(x = "Cover_Type", y= "Elevation", data = forest_train)
plt.show()

In [ ]:
# Reverse One hot encoding to get the variable name
wild_area = []
r,c = forest_train.shape

for row in range(0,r):
    for col in range(11,15):
        if forest_train.iloc[row,col] == 1:
            wild_area.append(forest_train.columns.values[col])
        

In [ ]:
forest_train['Wilderness_Area']=wild_area

In [ ]:
forest_train.tail()

In [ ]:
covertype_train = forest_train['Cover_Type']
data_train = forest_train.drop(['Id','Cover_Type','Wilderness_Area'],axis = 1)

In [ ]:
forest_test = pd.read_csv("/kaggle/input/forest-cover-type-prediction/test.csv")
data_test = forest_test.drop("Id",axis = 1)

In [ ]:
rtest,coltest = forest_test.shape
rtest

In [ ]:
rtest/rtrain

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier(n_estimators=10, criterion="gini", max_features="auto",bootstrap=True,oob_score=True,n_jobs = -1,random_state=42,verbose =1)


In [ ]:
classifier.fit(data_train,covertype_train)

In [ ]:
classifier.oob_score_

In [ ]:
pred_test = classifier.predict(data_test)  #Kaggle score 0.7218

In [ ]:
forest_test['Cover_Type'] = pred_test
forest_test[['Id','Cover_Type']].to_csv("ForestCover_rf_model.csv",index = False)

In [ ]:
classifier_final = RandomForestClassifier(n_estimators=110, criterion="gini", max_features=18,bootstrap=True,oob_score=True,n_jobs = -1,max_depth=30,random_state=42)
classifier_final.fit(data_train,covertype_train)

In [ ]:
classifier_final.oob_score_

In [ ]:
pred_test = classifier_final.predict(data_test) #Kaggle score 0.7558

In [ ]:
forest_test['Cover_Type'] = pred_test
forest_test[['Id','Cover_Type']].to_csv("ForestCover_rf_model.csv",index = False)

In [ ]:


class_weights = pd.DataFrame({"Class_count":forest_test['Cover_Type'].groupby(forest_test['Cover_Type']).agg(len)})
class_weights



In [ ]:
forest_train['Cover_Type'].groupby(forest_train['Cover_Type']).agg(len)
## In train data set all cover types have equal samples unlike test

In [ ]:
class_weights['Class_Weights'] = class_weights['Class_count']/sum(class_weights['Class_count'])
class_weights

In [ ]:
# Function to assign weights to the samples
def ret_weight(type):
    if type == 1:
        return 0.37
    elif type == 2:
        return 0.4
    elif type == 3:
        return 0.06
    elif type == 4:
        return 0.004
    elif type == 5:
        return 0.057
    elif type == 6:
        return 0.045
    else :
        return 0.056

In [ ]:
sample_weights = forest_train['Cover_Type'].apply(ret_weight)

In [ ]:
forest_train['Sample_weights'] = sample_weights

In [ ]:
forest_train.head()

In [ ]:
classifier_final.fit(data_train,covertype_train,sample_weight=sample_weights) 

In [ ]:
classifier_final.oob_score_

In [ ]:
pred_test = classifier_final.predict(data_test) # Kaggle score 0.7443

In [ ]:
forest_test['Cover_Type'] = pred_test
forest_test[['Id','Cover_Type']].to_csv("ForestCover_rf_model.csv",index = False)